<a href="https://colab.research.google.com/github/MicroprocessorX069/Image-Feature-Matching-and-Homography/blob/master/Copy_of_Project2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

In [0]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import random

def findKeypoints(input_name,output_name):
  img = cv.imread(input_name)
  gray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
  sift = cv.xfeatures2d.SIFT_create()
  kp, des = sift.detectAndCompute(gray,None)
  img=cv.drawKeypoints(gray,kp,img)
  #cv.imshow("Image", img)
  #cv.waitKey(0)
  #cv.destroyAllWindows()
  cv.imwrite(output_name,img)


def knnMatching(input_name1,input_name2,output_name):
  img1 = cv.imread(input_name1,0)          # queryImage
  img2 = cv.imread(input_name2,0) # trainImage
  # Initiate SIFT detector
  sift = cv.xfeatures2d.SIFT_create()
  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  # BFMatcher with default params
  bf = cv.BFMatcher()
  matches = bf.knnMatch(des1,des2, k=2)
  # Apply ratio test
  good = []
  for m,n in matches:
      if m.distance < 0.75*n.distance:
          good.append([m])
  img3=np.array([])       
  # cv.drawMatchesKnn expects list of lists as matches.
  img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,outImg=img3,flags=2)
  cv.imwrite(output_name,img3)
 
def calculateHomography(input_name1,input_name2):
  Min_matches=10
  img1=cv.imread(input_name1,0)
  img2=cv.imread(input_name2,0)

  sift=cv.xfeatures2d.SIFT_create()
  kp1, des1=sift.detectAndCompute(img1, None) # Here mask is None.
  kp2, des2=sift.detectAndCompute(img2, None)

  Flann_index=1
  #dict() is just a function to create a dictionary of arguments 
  #More arguments can be kept than required by the method

  index_param= dict(algorithm=Flann_index,trees=5)
  search_param=dict(checks=50)
  flann=cv.FlannBasedMatcher(index_param,search_param)
  matches=flann.knnMatch(des1,des2,k=2)
  good_matches=[]
  for m,n in matches:
    if m.distance<0.7*n.distance:
      good_matches.append(m)

  #Set a condition of n matches to find an object
  #If found, extract the locations, to find the perspective transform

  if(len(good_matches)>Min_matches):
    #queryIdx and trainIdx are the descriptors of the good matched locations
    src_pts=np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    dst_pts=np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    M,mask=cv.findHomography(src_pts,dst_pts,cv.RANSAC,5.0)
    return M, mask
  
def featureMatching(input_name1,input_name2,output_name):
  Min_matches=10
  img1=cv.imread(input_name1,0)
  img2=cv.imread(input_name2,0)

  sift=cv.xfeatures2d.SIFT_create()
  kp1, des1=sift.detectAndCompute(img1, None) # Here mask is None.
  kp2, des2=sift.detectAndCompute(img2, None)

  Flann_index=1
  #dict() is just a function to create a dictionary of arguments 
  #More arguments can be kept than required by the method

  index_param= dict(algorithm=Flann_index,trees=5)
  search_param=dict(checks=50)
  flann=cv.FlannBasedMatcher(index_param,search_param)
  matches=flann.knnMatch(des1,des2,k=2)
  good_matches=[]
  for m,n in matches:
    if m.distance<0.7*n.distance:
      good_matches.append(m)

  #Set a condition of n matches to find an object
  #If found, extract the locations, to find the perspective transform

  if(len(good_matches)>Min_matches):
    #queryIdx and trainIdx are the descriptors of the good matched locations
    src_pts=np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1,1,2)
    dst_pts=np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1,1,2)

    M,mask=cv.findHomography(src_pts,dst_pts,cv.RANSAC,5.0)
    matchesMask=mask.ravel().tolist()
    inliers=np.where(np.array(matchesMask)==1)[0]
    size_inliers=int(inliers.shape[0])
    matchesMask=np.zeros(np.array(matchesMask).shape)
    for i in range(0,10):
      random1=random.randint(0,size_inliers)
      matchesMask[inliers[random1]]=1
    matchesMask=matchesMask.ravel().tolist()
    h,w=img1.shape
    pts=np.float32([[0,0],[0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
    dst=cv.perspectiveTransform(pts,M)

    img2=cv.polylines(img2,[np.int32(dst)],True,255,3,cv.LINE_AA)

  else:
    print("not enoug matches")
    matchesMask=None

  draw_param=dict(matchColor=(0,255,0), singlePointColor=None, matchesMask= matchesMask, flags=2)
  imgQ4= cv.drawMatches(img1,kp1,img2,kp2,good_matches,None,**draw_param)
  cv.imwrite(output_name,imgQ4)

##Keypoint detection
Keypoints between two images are matched by identifying their nearest neighbours. But in some cases, the second closest-match may be very near to the first.

**sift.detect()** function finds the keypoint in the images. You can pass a mask if you want to search only a part of image. Each keypoint is a special structure which has many attributes like its (x,y) coordinates, size of the meaningful neighbourhood, angle which specifies its orientation, response that specifies strength of keypoints etc.

OpenCV also provides **cv.drawKeyPoints()** function which draws the small circles on the locations of keypoints.

Now to calculate the descriptor, OpenCV provides two methods.

1. Since you already found keypoints, you can call sift.compute() which computes the descriptors from the keypoints we have found. Eg: kp,des = sift.compute(gray,kp)
2. If you didn't find keypoints, directly find keypoints and descriptors in a single step with the function, sift.detectAndCompute().


In [0]:
#1.1
findKeypoints("mountain1.jpg","task1_sift1.jpg")
findKeypoints("mountain2.jpg","task1_sift2.jpg")

## Feature Matching using KNN

Brute-Force takes the descriptor of one feature in first set and is matched with all other features in second set using some distance calculation. And the closest one is returned.

For BF matcher, first we have to create the BFMatcher object using cv2.BFMatcher().

Second method returns k best matches where k is specified by the user. It may be useful when we need to do additional work on that.

Like we used cv2.drawKeypoints() to draw keypoints, cv2.drawMatches() helps us to draw the matches. It stacks two images horizontally and draw lines from first image to second image showing best matches. There is also cv2.drawMatchesKnn which draws all the k best matches. If k=2, it will draw two match-lines for each keypoint.

In [0]:
#1.2blob:https://colab.research.google.com/411a2ada-7cba-4ce3-bbd3-0c45350230c6
knnMatching("mountain1.jpg","mountain2.jpg","task1_matches_knn.jpg")

## Calculating homography matrix and masks

Homography matrix is a 3x3 matrix but with 8 DoF (degrees of freedom) as it is estimated up to a scale. It is generally normalized (see also 1) with h33=1 or h211+h212+h213+h221+h222+h223+h231+h232+h233=1.

FLANN stands for Fast Library for Approximate Nearest Neighbors. It contains a collection of algorithms optimized for fast nearest neighbor search in large datasets and for high dimensional features.

In [0]:
#1.3
M, mask= calculateHomography("mountain1.jpg","mountain2.jpg")
M

## Feature Matching 

FLANN stands for Fast Library for Approximate Nearest Neighbors. It contains a collection of algorithms optimized for fast nearest neighbor search in large datasets and for high dimensional features. It works more faster than BFMatcher for large datasets. We will see the second example with FLANN based matcher.

For FLANN based matcher, we need to pass two dictionaries which specifies the algorithm to be used, its related parameters etc

In [0]:
#1.4
featureMatching("mountain1.jpg","mountain2.jpg","task1_matches.jpg")

In [0]:
#Q1.5.
imgQ42 = cv.warpPerspective(img1, M, (img2.shape[1],img2.shape[0]))
cv.imwrite("task1 pano.jpg",imgQ42)